In [1]:
import pandas as pd

# Load EUTL dataset

## Check optimal dtypes
- based on number of unique values, set as category or string

In [16]:
df_eutl_2022 = pd.read_csv(r'../data/raw/transactions_eutl_2025/transactions_EUTL_PUBLIC_NOTESD_20251031.csv', nrows = 10000)

for col in df_eutl_2022.select_dtypes(include="object"):
    print(f"{col}: {df_eutl_2022[col].nunique(dropna=False) / len(df_eutl_2022)}")

TRANSACTION_ID: 0.7958
TRANSACTION_TYPE: 0.0004
TRANSACTION_DATE: 0.79
TRANSACTION_STATUS: 0.0001
TRANSFERRING_REGISTRY_NAME: 0.0033
TRANSFERRING_ACCOUNT_TYPE2: 0.0004
TRANSFERRING_ACCOUNT_TYPE3: 0.0007
TRANSFERRING_ACCOUNT_OPEN_DT: 0.144
TRANSFERRING_ACCOUNT_END_OF_VALIDITY: 0.0655
TRANSFERRING_ACCOUNT_NAME: 0.2351
TRANSFERRING_ACCOUNT_HOLDER: 0.1922
TRANSFERRING_ACCOUNT_HOLDER_ADDRESS1: 0.1836
TRANSFERRING_ACCOUNT_HOLDER_ADDRESS2: 0.0475
TRANSFERRING_ACCOUNT_HOLDER_CITY: 0.1294
TRANSFERRING_ACCOUNT_HOLDER_POSTAL_CODE: 0.1552
TRANSFERRING_ACCOUNT_HOLDER_COUNTRY_CODE: 0.0045
TRANSFERRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER: 0.1879
TRANSFERRING_ACCOUNT_HOLDER_LEI: 0.0574
TRANSFERRING_INSTALLATION_NAME: 0.1836
TRANSFERRING_INSTALLATION_PERMIT_IDENTIFIER: 0.1862
TRANSFERRING_INSTALLATION_PARENT_COMPANY: 0.0399
TRANSFERRING_INSTALLATION_SUBSIDIARY_COMPANY: 0.0111
TRANSFERRING_INSTALLATION_EPER_IDENTIFICATION: 0.0633
TRANSFERRING_INSTALLATION_CITY: 0.1476
TRANSFERRING_INSTALLATION_PO

C:\Users\ATapia\AppData\Local\Temp\ipykernel_28052\203569506.py:1: DtypeWarning: Columns (0: LULUCF_CODE_DESCRIPTION) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eutl_2022 = pd.read_csv(r'../data/raw/transactions_eutl_2025/transactions_EUTL_PUBLIC_NOTESD_20251031.csv', nrows = 10000)
C:\Users\ATapia\AppData\Local\Temp\ipykernel_28052\203569506.py:3: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  for col in df_eutl_2022.select_dtypes(include="object"):


In [17]:
dtypes = {
    # IDs / keys
    "TRANSACTION_ID": "string",

    # Low-cardinality / codes
    "TRANSACTION_TYPE": "category",
    "TRANSACTION_STATUS": "category",

    "TRANSFERRING_REGISTRY_NAME": "category",
    "ACQUIRING_REGISTRY_NAME": "category",
    "ORIGINATING_REGISTRY": "category",

    "TRANSFERRING_ACCOUNT_TYPE2": "category",
    "TRANSFERRING_ACCOUNT_TYPE3": "category",
    "ACQUIRING_ACCOUNT_TYPE2": "category",
    "ACQUIRING_ACCOUNT_TYPE3": "category",

    "TRANSFERRING_ACCOUNT_HOLDER_COUNTRY_CODE": "category",
    "ACQUIRING_ACCOUNT_HOLDER_COUNTRY_CODE": "category",

    "TRANSFERRING_INSTALLATION_MAIN_ACTIVITY": "category",
    "ACQUIRING_INSTALLATION_MAIN_ACTIVITY": "category",

    "UNIT_TYPE_DESCRIPTION": "category",
    "SUPP_UNIT_TYPE_DESCRIPTION": "category",
    "LULUCF_CODE_DESCRIPTION": "string",  # force away mixed types; you can later .astype("category") if you want

    # Lower-cardinality text (often worth category)
    "TRANSFERRING_INSTALLATION_PARENT_COMPANY": "category",
    "TRANSFERRING_INSTALLATION_SUBSIDIARY_COMPANY": "category",
    "ACQUIRING_INSTALLATION_PARENT_COMPANY": "category",
    "ACQUIRING_INSTALLATION_SUBSIDIARY_COMPANY": "category",

    "TRANSFERRING_ACCOUNT_HOLDER_LEI": "category",
    "ACQUIRING_ACCOUNT_HOLDER_LEI": "category",

    "TRANSFERRING_ACCOUNT_HOLDER_ADDRESS2": "category",
    "ACQUIRING_ACCOUNT_HOLDER_ADDRESS2": "category",
    "TRANSFERRING_INSTALLATION_ADDRESS2": "category",
    "ACQUIRING_INSTALLATION_ADDRESS2": "category",

    "TRANSFERRING_INSTALLATION_EPER_IDENTIFICATION": "category",
    "ACQUIRING_INSTALLATION_EPER_IDENTIFICATION": "category",

    # High-ish cardinality text -> string (safe default)
    "TRANSFERRING_ACCOUNT_NAME": "string",
    "TRANSFERRING_ACCOUNT_HOLDER": "string",
    "TRANSFERRING_ACCOUNT_HOLDER_ADDRESS1": "string",
    "TRANSFERRING_ACCOUNT_HOLDER_CITY": "string",
    "TRANSFERRING_ACCOUNT_HOLDER_POSTAL_CODE": "string",
    "TRANSFERRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER": "string",

    "TRANSFERRING_INSTALLATION_NAME": "string",
    "TRANSFERRING_INSTALLATION_PERMIT_IDENTIFIER": "string",
    "TRANSFERRING_INSTALLATION_CITY": "string",
    "TRANSFERRING_INSTALLATION_POSTAL_CODE": "string",
    "TRANSFERRING_INSTALLATION_ADDRESS1": "string",

    "ACQUIRING_ACCOUNT_NAME": "string",
    "ACQUIRING_ACCOUNT_HOLDER": "string",
    "ACQUIRING_ACCOUNT_HOLDER_ADDRESS1": "string",
    "ACQUIRING_ACCOUNT_HOLDER_CITY": "string",
    "ACQUIRING_ACCOUNT_HOLDER_POSTAL_CODE": "string",
    "ACQUIRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER": "string",

    "ACQUIRING_INSTALLATION_NAME": "string",
    "ACQUIRING_INSTALLATION_PERMIT_IDENTIFIER": "string",
    "ACQUIRING_INSTALLATION_CITY": "string",
    "ACQUIRING_INSTALLATION_POSTAL_CODE": "string",
    "ACQUIRING_INSTALLATION_ADDRESS1": "string",
}

## Set columns of interest and read

In [18]:
cols = ['TRANSACTION_ID', 'TRANSACTION_TYPE', 'TRANSACTION_DATE','TRANSFERRING_ACCOUNT_NAME',
       'TRANSFERRING_ACCOUNT_IDENTIFIER', 'TRANSFERRING_ACCOUNT_HOLDER', 'TRANSFERRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER',
       'TRANSFERRING_ACCOUNT_HOLDER_LEI', 'TRANSFERRING_INSTALLATION_NAME',
       'TRANSFERRING_INSTALLATION_INSTALLATION_IDENTIFIER','TRANSFERRING_INSTALLATION_PARENT_COMPANY',
       'TRANSFERRING_INSTALLATION_SUBSIDIARY_COMPANY','TRANSFERRING_INSTALLATION_MAIN_ACTIVITY', 'ACQUIRING_REGISTRY_NAME','ACQUIRING_ACCOUNT_NAME',
       'ACQUIRING_ACCOUNT_IDENTIFIER', 'ACQUIRING_ACCOUNT_HOLDER','ACQUIRING_ACCOUNT_HOLDER_LEI', 'ACQUIRING_INSTALLATION_NAME',
       'ACQUIRING_INSTALLATION_INSTALLATION_IDENTIFIER','ACQUIRING_INSTALLATION_PARENT_COMPANY',
       'ACQUIRING_INSTALLATION_SUBSIDIARY_COMPANY']

In [20]:
df_eutl_2022 = pd.read_csv(
    "../data/raw/transactions_eutl_2025/transactions_EUTL_PUBLIC_NOTESD_20251031.csv",
    usecols=cols,
    dtype=dtypes,
    parse_dates=["TRANSACTION_DATE"],
    low_memory=False
)

In [21]:
df_eutl_2022.info()

<class 'pandas.DataFrame'>
RangeIndex: 2142475 entries, 0 to 2142474
Data columns (total 22 columns):
 #   Column                                                   Dtype         
---  ------                                                   -----         
 0   TRANSACTION_ID                                           string        
 1   TRANSACTION_TYPE                                         category      
 2   TRANSACTION_DATE                                         datetime64[us]
 3   TRANSFERRING_ACCOUNT_NAME                                string        
 4   TRANSFERRING_ACCOUNT_IDENTIFIER                          float64       
 5   TRANSFERRING_ACCOUNT_HOLDER                              string        
 6   TRANSFERRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER  string        
 7   TRANSFERRING_ACCOUNT_HOLDER_LEI                          category      
 8   TRANSFERRING_INSTALLATION_NAME                           string        
 9   TRANSFERRING_INSTALLATION_INSTALLATION_IDENTIF